## Methodological Questions

### Question 1: Provide a general description of the contents of each of the datasets provided along with these instructions.  You can choose a software package you prefer to process the datasets.

We will conducting this case study in a jupyter notebook, using python. Specifically, we will be using pandas. The csvs are small enough to be easily imported into the dataframes, and pandas is perfect for data cleaning, manipulation and analysis.

General description of datasets:

1. Impressions Table: This is a central table in which all the other tables are connected to. It has the cookie level records as well as additional information, such as user id, impression type, ad properties, which contextualizes the impression.

2. Activity Table: This table is also a central table, in which the other tables are connected to as well. This table shows the most concrete actions that consumers take when interacting with an advertisement. It shows the conversion time and the last advertisement served before the customer interacts with the ad.

3. Placement table: This table gives more detailed information on the placement surrounding the impression. It provides explicit definitions, site placement, and cost.

4. Ad Table: This table is like a dictionary of all the different attributes of the ads. With this table, we know how each unique ad looked like, size, and purpose.

5. Market Table: This table is like a dictionary of all the different attributes of the market. With this table, we can look up the definitions of each unique consumer market.

6. OS Table: This table is like a dictionary of the different operating systems.

7. Site Table: This table is like a dictionary of all the different attributes of the sites. With this table, we understand the site’s category.


### Question 2: Describe the steps you have taken to convert the raw data into the dataset used in the analysis.

To convert the raw data (csvs) in to the dataset used in the analysis, I loaded them into panda dataframes. In addition, I change the format of the impression and conversion times to datetime.

In [3]:
import pandas as pd
import numpy as np

In [4]:
# Load all csvs in to pandas dataframes
activity = pd.read_csv('ACTIVITY.csv').drop(columns = ['Unnamed: 7','Unnamed: 8'])
ads = pd.read_csv('ADS.csv')
audience = pd.read_csv('AUDIENCE.csv')
impression = pd.read_csv('IMPRESSIONS.csv')
market = pd.read_csv('MARKET.csv')
placement = pd.read_csv('PLACEMENTS.csv')
os = pd.read_csv('OS.csv')
site = pd.read_csv('SITE.csv')

In [5]:
activity['Conversion_time'] = pd.to_datetime(activity['Conversion_time'])
impression['Impression_time'] = pd.to_datetime(impression['Impression_time'])

### Question 3: Describe some metrics you could create to help understand web behavior within the dataset.

There are many metrics that are essential in understanding campaign results and web behavior. Within the provided dataset, I believe the following are necessary to know:

1. Conversion per Impression (CPI): To evaluate the efficiency of the campaign, we need to understand if our resources are well spent. CPI shows if the method of advertising is effective to the relative amount of advertisements shown. 

    This metric can be applied to compare different channels and audiences to understand lead and traffic generation.


2. Cost per Action (and Conversion/Lead): Not all ads cost the same. If an advertisement is expensive, yet has an equal conversion rate as a cheaper alternative, it is imperative to know. Conversions are important, but not at an absorbent cost.

3. Ad frequency: How many times is the ad being shown to the same user? After a certain point, is it worth continue trying? What is the ideal number to show to a user? How often in a period of time is the user seeing the ad?

## Business Questions

### Question 1: Which Sites are most effective in generating conversions?  Which Sites are most efficient?  Why might this be?

To determine which sites are most 'effective' in generating conversion, we going to be looking only at the total number of conversions from each particular site. Most 'effective' refers to the site driving to the most conversion, regardless of other factors.

To determine which sites are most 'efficient' in generating conversion, I will be looking at the amount of conversions in reference to the number of impressions served. The most efficient sites would have a higher percentage of people converting per site.

In addition, efficiency can also be referring to the cost per conversion. We will be calculating this metric as well. 


In [6]:
# Most effective site
site_conversion = activity[['Conversion_time', 'User_id','Placement_id', 'Event_type'
                           ]].merge(placement[['Placement_id', 'Site_id']], how='left', on='Placement_id')
site_conversion = site_conversion.merge(site, how='left', on='Site_id')

In [7]:
site_conversion.groupby(['Site_id', 'Site_category']).count().sort_values('Conversion_time',
                                                                          ascending=False)['Conversion_time']

Site_id  Site_category
39362    Prog DSP         157
39869    Prog DSP         149
40362    Travel            27
47923    Travel            24
76442    News              22
44436    Social            17
18421    News              11
23421    Travel             7
29843    News               3
66727    Weather            3
99272    Gaming             3
23792    Weather            2
44439    Social             2
92700    Gaming             2
Name: Conversion_time, dtype: int64

According to this data, the site ids of 39362 and 39869, which are both categorized as Prog DSP, are the most effective in generating the conversion. Together, they make 70% of the total conversions.

The second question of most ‘effective’ is trickier. We have to match the conversion to the last served impression.
It is important to note the following:
 1. People are served the same advertisements impression (same placement, ad, and creative) before the conversion.
 2. Impression times are not the same as conversion time. We will place the particular impression that made the conversion as the one with the exact same attributes (same placement, ad, and creative) with the time closest, but not past, the conversion time.
We are essentially looking at impressions/conversion rate or more commonly known as conversion per impression (CPI) to test for efficiency.


In [8]:
# Merge conversing activity to corresponding impression

# Merging on direct IDs First
mdf = pd.merge(impression, activity, on = ['User_id', 'Placement_id', 'Ad_id', 'Creative_id'])

# Select the correct impression that led to conversion
def f(row):
    col = mdf[(mdf['User_id'] == row['User_id']) & 
              (mdf['Placement_id'] == row['Placement_id']) &
              (mdf['Ad_id'] == row['Ad_id']) &
              (mdf['Creative_id'] == row['Creative_id']) &
              (mdf['Impression_time'] < row['Conversion_time'])]['Impression_time'].max()
    return col

mdf = mdf[mdf['Impression_time'] == mdf.apply(f, axis=1)].reset_index(drop=True)

# Left join to original impression table
impression_to_conv = impression.merge(mdf, how = 'left', on = ['Impression_time', 'User_id', 'Placement_id', 
                                          'Ad_id', 'Creative_id', 'Operating_system_id',
                                          'Market_id'])

In [9]:
# Examples of impressions and conversion records by users:
impression_to_conv[(impression_to_conv['User_id'] == 'UID_fKLZ1GnT')]

,User_id,Impression_time,Placement_id,Ad_id,Creative_id,Operating_system_id,Market_id,Conversion_time,Event_type,Event_sub_type
1855,UID_fKLZ1GnT,2017-10-07 14:39:00,201277807,401164499,90754249,501012.0,3.0,2017-10-07 14:49:00,CONVERSION,POSTCLICK


In [10]:
impression_to_conv[(impression_to_conv['User_id'] == 'UID_kWXdO9aK')]

,User_id,Impression_time,Placement_id,Ad_id,Creative_id,Operating_system_id,Market_id,Conversion_time,Event_type,Event_sub_type
62618,UID_kWXdO9aK,2017-11-24 01:27:00,208429962,408153593,94233441,501012.0,7.0,NaT,NaN,NaN
140262,UID_kWXdO9aK,2017-12-24 15:36:00,208429962,408153593,94659761,501012.0,7.0,NaT,NaN,NaN
142131,UID_kWXdO9aK,2017-12-25 14:03:00,209618525,408943949,90606683,501012.0,7.0,NaT,NaN,NaN
142200,UID_kWXdO9aK,2017-12-25 14:52:00,208429962,408153593,94179716,501012.0,7.0,2017-12-26 05:46:00,CONVERSION,POSTVIEW


Now that we verified that our impression to conversion table is correct, we can continue with our analysis.

In [11]:
# Merge to Placement Table
site_i_c = impression_to_conv.merge(placement[['Placement_id', 'Site_id', ' CPM ']], how='left', on='Placement_id')
site_i_c = site_i_c.merge(site, how='left', on='Site_id')
# Get count of Impressions and Conversions by Site_id
site_cpi = site_i_c.groupby(['Site_id', 'Site_category']).count()[['Conversion_time', 'Impression_time']].reset_index()
# Get percentage of site conversion
site_cpi['cpi'] = site_cpi['Conversion_time']/site_cpi['Impression_time']
site_cpi.sort_values('cpi', ascending=False)

,Site_id,Site_category,Conversion_time,Impression_time,cpi
10,47923,Travel,24,2859,0.008395
11,66727,Weather,3,441,0.006803
7,40362,Travel,27,5553,0.004862
3,23792,Weather,2,529,0.003781
12,76442,News,22,6165,0.003569
2,23421,Travel,7,2326,0.003009
5,39362,Prog DSP,157,52490,0.002991
1,18421,News,11,3822,0.002878
4,29843,News,3,1098,0.002732
14,99272,Gaming,3,1297,0.002313


According to this table, the sites with the id 47923, 66727 and 40362 are the most efficient in conversions. It's interesting to see that categories of Travel and Weather are the top in conversion, especially for a traveling company's digital campaign.

In [12]:
# Add cost of each site advertising
site_i_c[' CPM '] = site_i_c[' CPM '].str.strip('$').astype(float)

In [13]:
site_i_c['cost'] = 1/site_i_c[' CPM ']
site_cost = site_i_c.groupby(['Site_id', 'Site_category']).sum()['cost'].reset_index()
site_cpi = site_cpi.merge(site_cost, on = ['Site_id', 'Site_category'])
site_cpi['cost_per_conversion'] = site_cpi['cost']/site_cpi['Conversion_time']

In [14]:
site_cpi.sort_values(by = 'cost_per_conversion')

,Site_id,Site_category,Conversion_time,Impression_time,cpi,cost,cost_per_conversion
12,76442,News,22,6165,0.003569,140.398389,6.381745
14,99272,Gaming,3,1297,0.002313,20.219609,6.739870
4,29843,News,3,1098,0.002732,23.647454,7.882485
10,47923,Travel,24,2859,0.008395,190.332922,7.930538
1,18421,News,11,3822,0.002878,92.010229,8.364566
7,40362,Travel,27,5553,0.004862,231.601602,8.577837
11,66727,Weather,3,441,0.006803,29.893323,9.964441
3,23792,Weather,2,529,0.003781,28.173811,14.086906
8,44436,Social,17,8818,0.001928,341.456826,20.085696
2,23421,Travel,7,2326,0.003009,151.537817,21.648260


In terms of cost efficiency, the site ids of 76442, 99272, and 29843 are the best. This may be due to the high volume of advertising space available. We also see that one of the Prog DSP category, (39869) is not cost efficient, as it takes an average of $128 per conversion

Althought DSP has higher overall effectiveness, it may not be always the best choice.  DSP's targets a wider range of audience, and naturally will bring in more conversions. In terms of impression efficiency, targeting weather and travel sites because these are sites that consumers are more likely than not going to check when thinking of travel.

### Question 2: Which Ads are most effective in generating conversions?  Which Ads are most efficient? Why might this be?

The definitions of 'effective' and 'efficient' will be conducted in the same manner as Question 1. 

In [15]:
# Most effective ad
ad_conversion = activity[['Conversion_time', 'User_id','Ad_id', 'Event_type'
                           ]].merge(ads, how='left')

In [16]:
ad_conversion.groupby(['Ad_id', 'Marketing_goal', 'Ad_format', 'Ad_size']).count().sort_values('Conversion_time',
                                                                          ascending=False)['Conversion_time']

Ad_id      Marketing_goal  Ad_format      Ad_size         
408153593  Brand           Standard       300x250             66
401164499  DR              Rich Media     300x250             39
401159678  DR              Standard       300x250             29
408887696  Brand           Video          :15                 26
408146836  Brand           Video          :15                 24
508428413  Brand           Video          :15                 24
401164505  DR              Standard       336x280             23
600949438  Brand           Video          :15                 20
408943949  Brand           Video          Various_Tracking    13
408887705  Brand           Video          :15                 11
408838410  Brand           Video          :15                 11
401163967  DR              Standard       728x90              10
400922607  Brand           Custom Social  Social 1             9
401163961  DR              Rich Media     300x250              9
408107373  Brand           Rich

In terms of effectiveness, it seems that the standard format reigns supreme. They are able to bring in the most conversions by numbers.

In [17]:
ads_i_c = impression_to_conv.merge(ads, how='left')
# Get count of Impressions and Conversions by Ad_id
ads_cpi = ads_i_c.groupby(['Ad_id', 'Marketing_goal', 'Ad_format', 'Ad_size']).count()[['Conversion_time', 'Impression_time']].reset_index()
# Get percentage of site conversion
ads_cpi['cpi'] = ads_cpi['Conversion_time']/ads_cpi['Impression_time']
ads_cpi.sort_values('cpi', ascending=False)

,Ad_id,Marketing_goal,Ad_format,Ad_size,Conversion_time,Impression_time,cpi
225,411215182,Brand,Video,:15,1,8,0.125000
93,408151198,Brand,Video,:15,2,49,0.040816
197,409365684,Brand,Standard,970x250,1,32,0.031250
228,411215194,Brand,Video,:15,4,146,0.027397
160,408841113,Brand,Video,:15,1,40,0.025000
80,408146836,Brand,Video,:15,24,1637,0.014661
16,401158616,DR,Rich Media,300x250,1,71,0.014085
87,408151096,Brand,Video,:15,1,72,0.013889
178,408906271,Brand,Video,:15,2,155,0.012903
172,408887696,Brand,Video,:15,26,2547,0.010208


However, when we evaluate efficiency, videos have the highest CPI. This may be due to video's inherently interactive nature or commanding attention from the consumer's media content.

In [18]:
ads_i_c = ads_i_c.merge(placement[['Placement_id', ' CPM ']], how = 'left', on = 'Placement_id')
ads_i_c[' CPM '] = ads_i_c[' CPM '].str.strip('$').astype(float)
ads_i_c['cost'] = 1/ads_i_c[' CPM ']
ads_cost = ads_i_c.groupby(['Ad_id', 'Marketing_goal', 'Ad_format', 'Ad_size']).sum()['cost'].reset_index()
ads_cpi = ads_cpi.merge(ads_cost, on = ['Ad_id', 'Marketing_goal', 'Ad_format', 'Ad_size'])
ads_cpi['cost_per_conversion'] = ads_cpi['cost']/ads_cpi['Conversion_time']
ads_cpi.sort_values(by = 'cost_per_conversion')

,Ad_id,Marketing_goal,Ad_format,Ad_size,Conversion_time,Impression_time,cpi,cost,cost_per_conversion
225,411215182,Brand,Video,:15,1,8,0.125000,0.488583,0.488583
93,408151198,Brand,Video,:15,2,49,0.040816,1.067015,0.533507
160,408841113,Brand,Video,:15,1,40,0.025000,0.589001,0.589001
197,409365684,Brand,Standard,970x250,1,32,0.031250,0.966181,0.966181
178,408906271,Brand,Video,:15,2,155,0.012903,2.417865,1.208933
172,408887696,Brand,Video,:15,26,2547,0.010208,40.722700,1.566258
228,411215194,Brand,Video,:15,4,146,0.027397,7.848576,1.962144
87,408151096,Brand,Video,:15,1,72,0.013889,2.310217,2.310217
80,408146836,Brand,Video,:15,24,1637,0.014661,64.208429,2.675351
230,508428413,Brand,Video,:15,24,2848,0.008427,73.812687,3.075529


According to the cost by conversion, video is also the most efficent advertising method as well. 

In the same vein of the site effectiveness and efficency, we see a difference between methods. The one that led to the greatest conversion was not necessarily the best way, as they did it with brute force. This data suggest heavily that the video campaigns were successful in efficency of impression and cost.

### Question 3: Which Audiences are most effective in generating conversions?  Which Audiences are most efficient?  Why might this be?

The definitions of 'effective' and 'efficient' will be conducted in the same manner as Question 1. 

In [19]:
audience_conversion = activity[['Conversion_time', 'User_id','Placement_id', 'Event_type'
                           ]].merge(placement[['Placement_id', 'Audience_id']], how='left', on='Placement_id')
audience_conversion = audience_conversion.merge(audience, how='left', on='Audience_id')

In [20]:
audience_conversion.groupby(['Audience_id', 'Audience_name']).count().sort_values('Conversion_time',
                                                                          ascending=False)['Conversion_time']

Audience_id  Audience_name      
529          Young Professionals    175
486          Frequent Travelers     123
743          Deal Seekers            67
356          Recently Retired        42
884          Adventure Junkies       19
114          Amtrak Users             3
Name: Conversion_time, dtype: int64

It seems that marketing to young professionals is most effective because they are the ones that signed up the most of this service.

In [21]:
# Merge to Placement Table
audience_i_c = impression_to_conv.merge(placement[['Placement_id', 'Audience_id', ' CPM ']], how='left', on='Placement_id')
audience_i_c = audience_i_c.merge(audience, how='left', on='Audience_id')
# Get count of Impressions and Conversions by Site_id
audience_cpi = audience_i_c.groupby(['Audience_id', 'Audience_name']).count()[['Conversion_time', 'Impression_time']].reset_index()
# Get percentage of site conversion
audience_cpi['cpi'] = audience_cpi['Conversion_time']/site_cpi['Impression_time']
audience_cpi.sort_values('cpi', ascending=False)

,Audience_id,Audience_name,Conversion_time,Impression_time,cpi
3,529,Young Professionals,175,63428,0.330813
0,114,Amtrak Users,3,6300,0.107143
4,743,Deal Seekers,67,30256,0.061020
2,486,Frequent Travelers,123,44427,0.052880
1,356,Recently Retired,42,11837,0.010989
5,884,Adventure Junkies,19,3849,0.000362


Young professionals also seem to be the best market for efficiency, as shown by the CPI rate. This may be due to young professional being more tech savvy to find the deals, as well as having a disposible income and an opportunistic attitude! 

In [22]:
# Add cost of each market advertising
audience_i_c[' CPM '] = audience_i_c[' CPM '].str.strip('$').astype(float)
audience_i_c['cost'] = 1/audience_i_c[' CPM ']
audience_cost = audience_i_c.groupby(['Audience_id', 'Audience_name']).sum()['cost'].reset_index()
audience_cpi = audience_cpi.merge(audience_cost, on = ['Audience_id', 'Audience_name'])
audience_cpi['cost_per_conversion'] = audience_cpi['cost']/audience_cpi['Conversion_time']
audience_cpi.sort_values(by = 'cost_per_conversion')

,Audience_id,Audience_name,Conversion_time,Impression_time,cpi,cost,cost_per_conversion
5,884,Adventure Junkies,19,3849,0.000362,186.861704,9.834827
1,356,Recently Retired,42,11837,0.010989,584.216285,13.909912
3,529,Young Professionals,175,63428,0.330813,5536.654921,31.638028
2,486,Frequent Travelers,123,44427,0.052880,7749.102134,63.000830
0,114,Amtrak Users,3,6300,0.107143,302.465431,100.821810
4,743,Deal Seekers,67,30256,0.061020,10321.501759,154.052265


Interestingly, adventure junkies have the lowest cost per conversion ratio. This may be due to an impulsive nature. This would make sense as deal seekers have the highest cost per conversion rate; they are more careful and do not jump at the first sight of a good deal. Young professional falls between the two, which makes sense in the level of impulsivity.

### Question 4: What are some possible assumptions or additional factors that may undermine your conclusions?

Possible assumptions that may undermine our conclusions include:

1. Conversion means retention: A conversion does not guarentee that the consumer stays in the program or invests in the program for long. For example, a young professional gets annoyed by the notifications and unsubscribes within the next day.

2. Conversion means active participant: Although the consumer may sign up for the program, they may turn off all notifications and never use the rewards program again.

### Question 5: What would you do to improve the analysis?  Is there additional data you’d like to have available that would allow you to sharpen your findings?

To improve my findings next time, I'd like integrate the three tables of site, audience, and advertisements together. I'm curious to see if there's a certain type of ad that works on young professional as opposed to deal seekers. In addition, I would like to see if the way the impression is delivered (by mobile or online) would make a difference between certain markets.

Additional data to sharpen my findings include the following:

1. Bounce rate - I want to know how many people visit the website and do not complete the conversion. Is there something we could do to push leads into conversions? In addition, how long do people stay on the sites, conversion or not?

2. Creative attributes- Certain creatives can resonate with certain audiences and drive them to action.

3. Retention - How long do people stay with the program after conversion. Do they leave after a day, a year? A person cancelling the service would not help the ending goal of the campaign.

4. Return on ad spend- It is clear that people do sign up for the program. However, the ultimate goal is to remind the consumers to buy more travel tickets/make more travel plans. How much did the campaign produce in profits afterwards?

5. Days until first purchase- After conversion, how long until the customer first makes their purchase from the program?


### Question 6: Optional

### Question 7: What would your recommendation to the client be for future campaigns?

Based on my findings, I'd recommend that the travel company invest in a few areas:
1. Advertise within weather and travel websites- there is a high efficiency in both conversions per impressions and cost per conversion.
2. Make more video advertisements- there is a high efficiency in both conversions per impressions and cost per conversion.
3. Market more towards Adventure Junkies and Young professionals- Adventure Junkies and Young Professionals are more likely to impulsively spend money than Deal Seekers.